Note: Not sure if the approach is working when only the complier LATE is identified, since we have many free parameters.
Hence, do the simulation for the sharp case.

- Maybe we can do a multistart from all $2^{d}$ corner points to avoid local maxima.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import optimagic as om
import pandas as pd
import plotly.graph_objects as go

from lp_relax.funcs.lp_relax import (
    _get_identification_kwargs,
    solve_lp_convex,
)

In [ ]:
num_points = 1000
start = 0.5
stop = 1

beta_grid = np.linspace(start, stop, num_points)

bfunc_type = "bernstein"  # "bernstein"

idestimands = "sharp"

k_bernstein = 5

shape_constraints = {
    # "shape_constraints": ("decreasing", "decreasing")
}

s_to_plot = [10, 20, 50, 100]

algorithm = "scipy_slsqp"

constraint_type = "sphere"

num_dims = (k_bernstein + 1) * 2

In [ ]:
def solve_model_for_grid_and_slope(s: float, beta_grid: np.ndarray) -> dict:
    value_fun = {}

    for beta in beta_grid:
        identification_kwargs = _get_identification_kwargs(
            beta=beta,
            bfunc_type=bfunc_type,
            shape_constraints=shape_constraints,
            idestimands=idestimands,
            k_bernstein=k_bernstein,
        )

        value_fun[beta] = solve_lp_convex(
            beta=beta,
            algorithm=algorithm,
            constraint_type=constraint_type,
            s=s,
            k_approximation=s,
            identification_kwargs=identification_kwargs,
        )

    return value_fun

In [ ]:
sols_by_s = [solve_model_for_grid_and_slope(s, beta_grid) for s in s_to_plot]

In [ ]:
# Put into single df
data = pd.concat(
    [
        pd.DataFrame(
            {
                "s": s,
                "beta": list(res.keys()),
                "lp": [res[beta]["lp"] for beta in res],
                "convex": [res[beta]["convex"] for beta in res],
            }
        )
        for s, res in zip(s_to_plot, sols_by_s, strict=False)
    ]
)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data[data["s"] == data["s"].unique()[0]]["beta"],
        y=data[data["s"] == data["s"].unique()[0]]["lp"],
        mode="lines",
        name="LP",
    )
)

for s in s_to_plot:
    data_select = data[data["s"] == s]
    fig.add_trace(
        go.Scatter(
            x=data_select["beta"],
            y=data_select["convex"],
            mode="lines",
            name=f"Convex, s={s}",
        )
    )


# Set y range 0, 1
fig.update_yaxes(range=[0.2, 1])
fig.update_xaxes(range=[0.5, 0.95])

fig.show()

In [ ]:
beta = 0.8

identification_kwargs = _get_identification_kwargs(
    beta=beta,
    bfunc_type="bernstein",
    shape_constraints=shape_constraints,
    idestimands="sharp",
    k_bernstein=11,
)

om_opt = solve_lp_convex(
    beta=beta,
    algorithm="scipy_slsqp",
    constraint_type="sphere",
    s=1,
    k_approximation=100,
    identification_kwargs=identification_kwargs,
    return_optimizer=True,
)

In [ ]:
res = om_opt(algorithm="scipy_slsqp")
res

In [ ]:
om.criterion_plot(res)